# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [3]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, deque
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import cPickle as pickle
import slate 
import re
from fenaroli_parser.parser import text_from_pdf, make_ingredient_compound_dict

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [5]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [6]:
cleaned_joined = joined.replace('.', ' ')

## RE for flavor compounds and ingredients

## Here I have the material I need to build the ingredient:compound mapping

In [175]:
def flav_ingr_mapper(raw_text):
    
    flavor_dict = defaultdict(list)
    ignore_terms = ['EINECS No :', 'EINECS No ']
    re.MULTILINE
    ingr_pattern = re.compile('Natural occurrence:(.+)')
    comp_pattern = re.compile('(.*[A-Z]{6,20}.*)')
    
    blobs = raw_text.split('\n\n')
    comp_q = deque([], maxlen=4)
    comp_list = []
    ing = None
    comp = None
    for i in range(len(blobs)):
        text = blobs[i].replace('\n', '')
        potential_ing = re.findall(ingr_pattern, text)
        potentiacomp = re.findall(comp_pattern, text)
        if comp and comp[0] not in ignore_terms:
            try:
                if comp_q[-1] != comp[0]:
                    comp_q.append(comp[0])
                    comp_list.append(comp[0])
            except IndexError:
                comp_q.append(comp[0])
                comp_list.append(comp[0])

        if ing:
            try:
                flavor_dict[comp_q.pop().strip()] = ing[0].strip()
            except IndexError:
                pass
            
    return comp_list, flavor_dict

In [231]:
re.MULTILINE
ingredient_pattern = re.compile('Natural\s+occurrence:(.+)')

blobs = cleaned_joined.split('\n\n')
comp_list = []
for i in range(int(len(blobs)*0.2), len(blobs)):
    text = blobs[i].replace('\n', '')
    potential_ing = re.findall(ingredient_pattern, text)
    if potential_ing:
        comp_list.append(potential_ing[0].strip())

In [12]:
flav_comp_dict = text_from_pdf(cleaned_joined)

In [13]:
# print len(set(comp_list))
# print len(comp_list)
print len(flav_comp_dict.keys())

982


In [14]:
flav_comp_dict['ISOBUTYL 2-BUTENOATE']

flav_comp_dict['ISOBUTYL BUTYRATE']

flav_comp_dict['ISOBUTYL CINNAMATE']

flav_comp_dict['TRIACETIN']

'Reported found in papaya'

This is the compound - ingredient mapping I've been striving for

In [320]:
with open('./flavor_compund_map.pkl', 'w') as f:
    pickle.dump(flav_comp_dict, f)

In [66]:
ing_comp_dict = make_ingredient_compound_dict(flav_comp_dict)

In [60]:
for key, val in ing_comp_dict.iteritems():
    print key, val, '\n'

macadamia nuts ['ISOBUTYRALDEHYDE', 'ISOPHORONE', 'HEPTYL ALCOHOL'] 

mackerel ['DISODIUM 5\xca\xb9-INOSINATE'] 

yellow ['PHENETHYL BUTYRATE', 'cis-3-HEXEN-1-YL ACETATE', 'ISOAMYL BUTYRATE'] 

naranjilla fruit (solanum quitense lam ) ['BUTAN-3-ONE-2-YL BUTANOATE'] 

asian ['CINNAMIC ACID'] 

grapefruit peel oil ['p-MENTHAN-2-OL', 'UNDECYL ALCOHOL', '4\xca\xb9-METHYLACETOPHENONE', '\xce\xb2-PINENE'] 

mandarin) to as low as 1% (palmarosa) ['LIMONENE (d-, l-, and dl-)'] 

black chokeberry ['cis-3-HEXENYL FORMATE', 'DIHYDRO-\xce\xb2-IONONE'] 

the product obtained from natural sources is probably levorotatory ['p-\xce\xb1-DIMETHYLBENZYL ALCOHOL'] 

hyssop ['4-(2,6,6-TRIMETHYL-CYCLOHEXA-1,3-DIENYL)BUT-2-EN-4-ONE', '6,10-DIMETHYL-5,9-UNDECADIEN-2-ONE'] 

l ) ['ACETANISOLE', 'ACETANISOLE', 'ALLYL DISULFIDE', 'BENZALDEHYDE', 'METHYL NICOTINATE'] 

it exhibits a positive optical rotation ['CEDARWOOD OIL ALCOHOLS'] 

blackberries ['5-HYDROXY-2-DECENOIC ACID \xce\xb4-LACTONE', '2,5-DIMETHYL-4-M

In [67]:
ingredient_list = ing_comp_dict.keys()

In [68]:
ingredient_list.sort(key = lambda s: len(s))

In [69]:
ingredient_list

['fig',
 'cow',
 'cod',
 'ham',
 'hop',
 'gin',
 'fat',
 'tea',
 'rye',
 'egg',
 'pea',
 'rum',
 'eel',
 'jam',
 'soy',
 'oak',
 'mate',
 'figs',
 'blue',
 'tolu',
 '(raw',
 'wort',
 'root',
 'nuts',
 'over',
 'meat',
 'bone',
 'navy',
 'kiwi',
 'seed',
 'mint',
 'sage',
 'rice',
 'beef',
 'been',
 'beer',
 'tuna',
 'kale',
 'kelp',
 'okra',
 'peel',
 'corn',
 'plum',
 'bark',
 'form',
 'vivo',
 'more',
 'leek',
 'milk',
 'pork',
 'port',
 'sake',
 'wood',
 'peru',
 'sour',
 'goat',
 'oats',
 'seal',
 'wine',
 '53%)',
 'many',
 'rose',
 'clam',
 'virg',
 'buds',
 'pear',
 'malt',
 'food',
 'hook',
 'fish',
 'raw)',
 'also',
 'most',
 'mace',
 'br )',
 'eggs',
 'ghee',
 'used',
 'some',
 'palm',
 'lard',
 'stem',
 'dill',
 'lime',
 'from',
 'crab',
 'oils',
 'lamb',
 'feta',
 'veal',
 'lily',
 'hops',
 'bean',
 'jasm',
 'rums',
 'wild',
 'balm',
 'well',
 'leaf',
 'lean',
 'peas',
 'hosp',
 'pike',
 'asian',
 'buchu',
 'bacon',
 'meats',
 'shoyu',
 'myrrh',
 'pecan',
 'small',
 'honey',

In [72]:
with open('./data/first_ing_comp_dict.pkl', 'w') as f:
    pickle.dump(ing_comp_dict, f)